Hydrous DB importer / manager
Writen by LW/ES 20171224

Declarations and imports

In [1]:
import pandas
import os
import glob2
import json
import io
import piexif

declare globals and init variables

In [34]:
rootdir = '/Users/lucas/Documents/hydrousdb/hyrdousdb/data'
searchdir = rootdir + '/**/*'
twod_extention_list = ['.jpg','.png']
threed_extention_list = ['.obj', '.stl']
classignorelist = ['process','importfromtext','extract_exif']

declare classes

testjson = {'projname': 'testfile',
           'date' : '20171227',
           'location' : 'Maldives',http://localhost:8888/notebooks/hyrdousdb/db_process.ipynb#
           'photographer' : 'Elle Stapleton',
           'species' : 'Cool Coral',
            'latitude' : 0.216724, 
            'longitude' : 73.185691
           }
json_string = json.dumps(testjson)
fart = pandas.Series(testjson)
fart
fart.to_json('data.json')

In [ ]:
# static functions
os.getcwd()

In [117]:
class commondata(object):
    def __iter__(self):
        for attr, value in self.__dict__.iteritems(): #only return indicies
            yield attr, value
    def __str__(self):
        return "This is data common to all data sources"
    def __init__(self):
        #input from text file, init to blank
        self.projname = ''
        self.date = ''
        self.location = ''
        self.latitude = ''
        self.longitude = ''
        self.photographer = ''
        self.species = ''
        self.importedkeylist = []
        
        #passed from program
        self.filename = ''
        self.file_extension = ''
        self.fileroot = ''


    def importfromtext(self, import_textfile): 
        for key in [a for a in dir(import_textfile) if not a.startswith('__')]:
            if key in [a for a in dir(self) if not a.startswith('__')]:
                if key in self.importedkeylist:
                    setattr(self, key, getattr(import_textfile, key))

class info_text_file(commondata):
    def __str__(self):
        return "This is a metadata object, which inherets a lot of parameters from commondata class."
    def __init__(self, filename, fileroot):
        commondata.__init__(self)
        self.filename = filename
        self.fileroot = fileroot        
    def process(self):

        print 'found json file ' + self.filename +', importing values'
        with open(self.filename, 'r') as f:
            datastore = json.load(f)
        #this might be more effecient if we make a common function which iterates through keys
        for key in datastore:
            if key in [a for a in dir(self) if not a.startswith('__')]:
                #I think the problem is here - this doesn't appear to be working. I'm not sure why. check setattr method. 
                setattr(self, key, datastore[key]) #sets attribute to value in json import
                print a
                self.importedkeylist.append(key)
        print '***************imported key list****************'
        print self.importedkeylist
      
class twod_img(commondata):
    def __str__(self):
        return "This is a 2d image object, which inherets a lot of parameters from commondata class."
    def __init__(self, filename, file_extension, fileroot):
        commondata.__init__(self)
        self.filename = filename
        self.file_extension = file_extension
        self.fileroot = fileroot
        self.size = os.path.getsize(self.filename)
        self.resolution = []
        self.colorcorrected_image = False
    def extract_exif(self):
        exif_dict = piexif.load(self.filename)
        #dump(exif_dict) #- Get exif as bytes to save with JPEG.
        #insert(exif_bytes, filename) #- Insert exif into JPEG.
        #remove(filename) #- Remove exif from JPEG.)

        #extract exif data from 2d image
    def vomit_values(self):
        returnlist = []
        returndict = {}
        for a in [b for b in dir(self) if not b.startswith('__')]:
            returnval = getattr(self, a) #sets attribute to value in json import
            print returnval
            returndict = [a,returnval]
            returnlist.append(returndict)
        return returnlist

class threed_model(commondata):
    def __str__(self):
        return "This is a 3d model object, which inherets a lot of parameters from commondata class."
    def __init__(self, filename, file_extension, fileroot):
        commondata.__init__(self)
        self.filename = filename
        self.file_extension = file_extension
        self.fileroot = fileroot
        self.size = os.path.getsize(filename)
    #def collect_files(self):
        #create associative link between texture, 3d, and other files of same model (.mtl, .obj, .jpg)

In [118]:
import_dict = {}
json_identifier_dict = {}

for f in glob2.iglob(searchdir + '.json'):
    fileroot = os.path.dirname(f)
    if fileroot in json_identifier_dict:
        print'WARNING: multiple text files found in folder, ignoring ' + f
        pass
    if fileroot not in json_identifier_dict :    
        json_identifier_dict[f] = info_text_file(f, fileroot)
        json_identifier_dict[f].process() #process the metadata
#need to check for nested .json files to avoid problems.
#create copy of json identifier dicts so it can be modified - dicts are immutable
json_identifier_dict_cleaned = json_identifier_dict.copy()

for key1 in json_identifier_dict:
    for f in glob2.iglob(json_identifier_dict[key1].fileroot + '/*/**/*' + '.json'):
        json_identifier_dict_cleaned.pop(f)
        print'WARNING: json file found in subdirectory. Ignoring ' + f
        pass
        
for key in json_identifier_dict_cleaned:
    for f in glob2.iglob(json_identifier_dict_cleaned[key].fileroot + '/**/*'):
        filename, file_extension = os.path.splitext(f)
        fileroot = os.path.dirname(f)
        if file_extension in ['.json']:
            print'WARNING: json file found in subdirectory. Ignoring ' + f
            pass
        if file_extension in twod_extention_list:
            import_dict[f] = twod_img(f, file_extension, fileroot)
            import_dict[f].importfromtext(json_identifier_dict_cleaned[key])
            pass
        if file_extension in threed_extention_list:
            import_dict[f] = threed_model(f, file_extension, fileroot)
            import_dict[f].importfromtext(json_identifier_dict_cleaned[key])
            pass
        if file_extension not in threed_extention_list + twod_extention_list:
            print 'WARNING: Filetype not recognized, ignoring ' + f
            pass
        else:
            pass
fart = '/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6242.jpg'
print import_dict[fart].filename
import_dict[fart].vomit_values()


found json file /Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/data copy.json, importing values
species
species
species
species
species
species
species
***************imported key list****************
[u'projname', u'photographer', u'longitude', u'latitude', u'location', u'date', u'species']
found json file /Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/data.json, importing values
species
species
species
species
species
species
species
***************imported key list****************
[u'projname', u'photographer', u'longitude', u'latitude', u'location', u'date', u'species']
found json file /Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/data.json, importing values
species
species
species
species
species
species
species
***************imported key list****************
[u'projname', u'photographer', u'longitude', u'latitude', u'location', u'date', u'species']
/Users/lucas/Documents/hydrousdb/hyrdousd

[['colorcorrected_image', False],
 ['date', ''],
 ['extract_exif',
  <bound method twod_img.extract_exif of <__main__.twod_img object at 0x10afecc90>>],
 ['file_extension', '.jpg'],
 ['filename',
  '/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6242.jpg'],
 ['fileroot',
  '/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw'],
 ['importedkeylist', []],
 ['importfromtext',
  <bound method twod_img.importfromtext of <__main__.twod_img object at 0x10afecc90>>],
 ['latitude', ''],
 ['location', ''],
 ['longitude', ''],
 ['photographer', ''],
 ['projname', ''],
 ['resolution', []],
 ['size', 12750883],
 ['species', ''],
 ['vomit_values',
  <bound method twod_img.vomit_values of <__main__.twod_img object at 0x10afecc90>>]]

In [114]:
#FIHURE OUT why the fuck the parameters are not sticking for size... and why there is a folder in the output list.

pandas_dict = []
attribute_list = {}


for key in import_dict:
    #print 'import_dict'
    #print getattr(import_dict[key],'size')
    attribute_index = [a for a in dir(import_dict[key]) if not a.startswith('__')]
    #print attribute_index
    for attribute in attribute_index:
        print attribute
        print key
        print getattr(import_dict[key],attribute)
    for attribute_index in [a for a in dir(import_dict[key]) if not a.startswith('__')]:
        print attribute_index
        print getattr(import_dict[key],attribute_index)
        fart = import_dict[key]
        print getattr(fart,attribute_index)
        if attribute_index not in classignorelist:
            attribute_list[attribute_index] = getattr(import_dict[key], attribute_index)
            print attribute_list[attribute_index]
            pandas_dict.append(attribute_list)
    

    #print attribute_list.values()
    #print attribute_list['size']
    #print ' '
    #print 'FART'


#print pandas_dict
        
#print pandas_dict['/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6315.jpg']
#print pandas_dict[0].values()
#print ''
#print pandas_dict[40].values()
pandas.DataFrame.from_dict(pandas_dict)


colorcorrected_image
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6242.jpg
False
date
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6242.jpg

extract_exif
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6242.jpg
<bound method twod_img.extract_exif of <__main__.twod_img object at 0x10b1cae90>>
file_extension
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6242.jpg
.jpg
filename
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6242.jpg
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6242.jpg
fileroot
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6242.jpg
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw
importedkeylist
/Users/lucas/Doc


location
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6253.jpg

longitude
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6253.jpg

photographer
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6253.jpg

projname
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6253.jpg

resolution
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6253.jpg
[]
size
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6253.jpg
12222909
species
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6253.jpg

vomit_values
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6253.jpg
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1ca910>



projname



resolution
[]
[]
[]
size
10247084
10247084
10247084
species



vomit_values
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1bb610>>
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1bb610>>
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1bb610>>
colorcorrected_image
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6322.jpg
False
date
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6322.jpg

extract_exif
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6322.jpg
<bound method twod_img.extract_exif of <__main__.twod_img object at 0x10b1bbb10>>
file_extension
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6322.jpg
.jpg
filename
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6322.jpg
/Users/lucas

size
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6271.jpg
12044516
species
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6271.jpg

vomit_values
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6271.jpg
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1ca4d0>>
colorcorrected_image
False
False
False
date



extract_exif
<bound method twod_img.extract_exif of <__main__.twod_img object at 0x10b1ca4d0>>
<bound method twod_img.extract_exif of <__main__.twod_img object at 0x10b1ca4d0>>
file_extension
.jpg
.jpg
.jpg
filename
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6271.jpg
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6271.jpg
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6271.jpg
f



location



longitude



photographer



projname



resolution
[]
[]
[]
size
10517121
10517121
10517121
species



vomit_values
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1bbf50>>
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1bbf50>>
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1bbf50>>
colorcorrected_image
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6261.jpg
False
date
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6261.jpg

extract_exif
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6261.jpg
<bound method twod_img.extract_exif of <__main__.twod_img object at 0x10b1ca810>>
file_extension
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw/20171211-_32B6261.jpg
.jpg
filename
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Mode

/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/1-Model-Verticle wall_jpodgs/raw
importedkeylist
[]
[]
[]
importfromtext
<bound method twod_img.importfromtext of <__main__.twod_img object at 0x10b1ca190>>
<bound method twod_img.importfromtext of <__main__.twod_img object at 0x10b1ca190>>
latitude



location



longitude



photographer



projname



resolution
[]
[]
[]
size
14306617
14306617
14306617
species



vomit_values
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1ca190>>
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1ca190>>
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1ca190>>
colorcorrected_image
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6301.jpg
False
date
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6301.jpg

extract_exif

longitude



photographer



projname



resolution
[]
[]
[]
size
13950851
13950851
13950851
species



vomit_values
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1ca8d0>>
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1ca8d0>>
<bound method twod_img.vomit_values of <__main__.twod_img object at 0x10b1ca8d0>>
colorcorrected_image
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6323.jpg
False
date
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6323.jpg

extract_exif
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6323.jpg
<bound method twod_img.extract_exif of <__main__.twod_img object at 0x10b1bbf10>>
file_extension
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/20171211-_32B6323.jpg
.jpg
filename
/Users/lucas/Documents/hydrousdb/hyrdousdb/data/2-Model-Blue soft coral-jpgs/2017121

,colorcorrected_image,date,file_extension,filename,fileroot,importedkeylist,latitude,location,longitude,photographer,projname,resolution,size,species,vomit_values
0,False,,.jpg,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,[],,,,,,[],13259896,,<bound method twod_img.vomit_values of <__main...
1,False,,.jpg,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,[],,,,,,[],13259896,,<bound method twod_img.vomit_values of <__main...
2,False,,.jpg,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,[],,,,,,[],13259896,,<bound method twod_img.vomit_values of <__main...
3,False,,.jpg,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,[],,,,,,[],13259896,,<bound method twod_img.vomit_values of <__main...
4,False,,.jpg,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,[],,,,,,[],13259896,,<bound method twod_img.vomit_values of <__main...
5,False,,.jpg,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,[],,,,,,[],13259896,,<bound method twod_img.vomit_values of <__main...
6,False,,.jpg,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,[],,,,,,[],13259896,,<bound method twod_img.vomit_values of <__main...
7,False,,.jpg,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,[],,,,,,[],13259896,,<bound method twod_img.vomit_values of <__main...
8,False,,.jpg,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,[],,,,,,[],13259896,,<bound method twod_img.vomit_values of <__main...
9,False,,.jpg,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,/Users/lucas/Documents/hydrousdb/hyrdousdb/dat...,[],,,,,,[],13259896,,<bound method twod_img.vomit_values of <__main...


In [ ]:
filepath = '/Users/lucas/Documents/hydrousdb/hyrdousdb/data/3-Model-Black coral-jpgs/20171211-_32B6368.jpg'
exif_dict = piexif.load(filepath)
for key in exif_dict:
    print key
print exif_dict

o = io.BytesIO()
thumb_im = Image.open("foo.jpg")
thumb_im.thumbnail((50, 50), Image.ANTIALIAS)
thumb_im.save(o, "jpeg")
thumbnail = o.getvalue()

zeroth_ifd = {piexif.ImageIFD.Make: u"Canon",
              piexif.ImageIFD.XResolution: (96, 1),
              piexif.ImageIFD.YResolution: (96, 1),
              piexif.ImageIFD.Software: u"piexif"
              }
exif_ifd = {piexif.ExifIFD.DateTimeOriginal: u"2099:09:29 10:10:10",
            piexif.ExifIFD.LensMake: u"LensMake",
            piexif.ExifIFD.Sharpness: 65535,
            piexif.ExifIFD.LensSpecification: ((1, 1), (1, 1), (1, 1), (1, 1)),
            }
gps_ifd = {piexif.GPSIFD.GPSVersionID: (2, 0, 0, 0),
           piexif.GPSIFD.GPSAltitudeRef: 1,
           piexif.GPSIFD.GPSDateStamp: u"1999:99:99 99:99:99",
           }
first_ifd = {piexif.ImageIFD.Make: u"Canon",
             piexif.ImageIFD.XResolution: (40, 1),
             piexif.ImageIFD.YResolution: (40, 1),
             piexif.ImageIFD.Software: u"piexif"
             }

exif_dict = {"0th":zeroth_ifd, "Exif":exif_ifd, "GPS":gps_ifd, "1st":first_ifd, "thumbnail":thumbnail}
exif_bytes = piexif.dump(exif_dict)
im = Image.open("foo.jpg")
im.thumbnail((100, 100), Image.ANTIALIAS)
im.save("out.jpg", exif=exif_bytes)

In [ ]:
for ifd_name in exif_dict:
    print("\n{0} IFD:".format(ifd_name))
    for key in exif_dict[ifd_name]:
        try:
            print(key, exif_dict[ifd_name][key][:10])
        except:
            print(key, exif_dict[ifd_name][key])